# Rainfall regression mapped to ground motion points

As the title says. This shows the results of the regression on the rainfall timeseries at the points where we have ground motion data. 

Given the different resolutions we only have two regression values because all the ground motion points are within 2 pixels of the rainfall data

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ipywidgets import *


In [ ]:
rain_csv = pd.read_csv('rainfall_timeseries_data.csv')
rain_csv.rename(columns={rain_csv.columns[0]: "date" }, inplace = True)
rain_csv.set_index('date', inplace=True)

In [ ]:
rain_reg = pd.read_csv('rainfall_regression_values.csv')
rain_reg.set_index('pid', inplace=True)

In [ ]:
# get the widget working 
full_gm_data = pd.read_csv('./tmp/160-IW1-414-s1-asc1-v2020.csv')
full_gm_data.rename(columns=lambda x: x.strip(), inplace=True)

In [ ]:
gm_lat = full_gm_data['latitude']
gm_lon = full_gm_data['longitude']
pid = full_gm_data['pid']

In [ ]:
gm_lat_lon = pd.DataFrame({'latitude':gm_lat, 'longitude':gm_lon})
gm_lat_lon.index = pid

In [ ]:
rain_reg['latitude'] = gm_lat_lon['latitude']
rain_reg['longitude'] = gm_lat_lon['longitude']

In [ ]:
mapping = {'ascending': 1, 'descending': 2, 'constant': 3, 'irregular': 4}

rain_reg.replace({'reg_type': mapping}, inplace=True)

In [ ]:
rain_reg.head()

In [ ]:
import folium
import branca.colormap as cm
#create a map
this_map = folium.Map(prefer_canvas=True)

#create a color map
color_var = 'slope' #what variable will determine the color
cmap = cm.LinearColormap(['cyan', 'red'],
                         vmin=rain_reg[color_var].min(), vmax=rain_reg[color_var].max(),
                         caption = color_var)

#Add the color map legend to your map
this_map.add_child(cmap)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
    folium.CircleMarker(location=[point.latitude, point.longitude],
                        fill_color=cmap(point[color_var]),
                        radius=2,
                        weight=0).add_to(this_map)

#use df.apply(,axis=1) to iterate through every row in your dataframe
rain_reg.apply(plotDot, axis = 1)

#Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())

#Save the map to an HTML file
this_map.save(os.path.join('figures/rainfall_points_reg.html'))


this_map

